# 모듈 임포트

In [1]:
# Change Directory to everytime base Folder
import os
cwd = os.getcwd()
os.chdir('/Users/min-kyulee/Desktop/와플스튜디오/team5-server/everytime/')

# Django Setup
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

# Shell Plus Model Imports
from board.models import BestBoard, Board, HotBoard
from comment.models import Comment
from django.contrib.admin.models import LogEntry
from django.contrib.auth.models import Group, Permission
from django.contrib.contenttypes.models import ContentType
from django.contrib.sessions.models import Session
from django.contrib.sites.models import Site
from post.models import Post, PostImage, PostTag, Tag, UserPost
from rest_framework.authtoken.models import Token
from user.models import SocialAccount, User

# Shell Plus Django Imports
from django.core.cache import cache
from django.conf import settings
from django.contrib.auth import get_user_model
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When
from django.utils import timezone
from django.urls import reverse
from django.db.models import Exists, OuterRef, Subquery

# import Library
import numpy as np
import pandas as pd

os.chdir(cwd)

# 함수 정의

In [15]:
# json 형식으로 변환시키기 전 int값들의 타입을 numpy.int64 -> int로 전처리
def intDecoder(dic):
    for key, value in dic.items():
        if type(value) == np.int64:
            dic[key] = int(value)

# 최대길이의 값과 길이를 출력
def printMax(arr):
    maxlen = 0
    maxitem = ''
    for item in arr:
        if len(item) > maxlen:
            maxlen = len(item)
            maxitem = item
    print('최대길이', maxlen, maxitem)

# 장고 CHIOCE FIELD 용 문자열 생성
def printTuple(items):
    for item in items:
        print(f"\t\t('{item}', '{item}'),")

# Blank 값 처리
def removeNaN(items):
    if 'nan' in items:
        print('null is True')
        items = np.delete(items, np.where(items=='nan'))
    else:
        print('null is False')
        
# 특정 열에서 중복되는 값 제거
def getUniqueList(data_frame, col_name):
    items = np.array(lecturelist[col_name], dtype=str)
    items = np.unique(items)
    removeNaN(items)
    return items

# lecture key 수정
def reviseKey(lecture):
    en_set = ['title', 'instructor', 'college', 'department', 'classification', 
              'degree', 'grade', 'course_code', 'lecture_code', 'credits', 'lecture', 'laboratory', 
              'type', 'locations', 'cart', 'quota', 'remark', 'language', 'times']
    kr_set = ['교과목명', '주담당교수', '개설대학', '개설학과', '교과구분', '이수과정', 
              '학년', '교과목번호', '강좌번호', '학점', '강의', '실습', 
              '수업형태', '강의실(동-호)(#연건, *평창)', '장바구니신청', '정원', '비고', '강의언어', '수업교시']
    for en, kr in zip(en_set, kr_set):
        lecture[en] = lecture.pop(kr)
        
# Nan 값을 인식 못 하므로 None으로 수정
def NaNtoNone(lecture):
    nanlist = []
    for key, value in lecture.items():
        if str(value) == 'nan':
            lecture[key] = None
            
# 언어와 형태 비고에 포함
def reviseRemark(lecture):
    language = lecture.pop('language')
    lecture_type = lecture.pop('type')
    remark = lecture.pop('remark')
    if remark is not None:
        if language != '한국어' and lecture_type is not None:
            lecture['remark'] = remark + '\n' + '언어:' + language + ', ' + '형태:' + lecture_type
        elif 'type' in lecture:
            lecture['remark'] = remark + '\n' + '형태:' + lecture_type
        elif language != '한국어':
            lecture['remark'] = remark + '\n' + '언어:' + language
    else:
        if language != '한국어' and lecture_type is not None:
            lecture['remark'] = '언어:' + language + ', ' + '형태:' + lecture_type
        elif '수업형태' in lecture:
            lecture['remark'] = '형태:' + lecture_type
        elif language != '한국어':
            lecture['remark'] = '언어:' + language


# 데이터 불러오기 및 전처리

## pandas Data Frame 불러오기

In [16]:
lecturelist = pd.read_excel("./수강편람.xls", header=2)
# lecturelist

## 데이터 전처리

In [17]:
lectures = []
for i in range(len(lecturelist)):
    '''
    장바구니신청 -> 담은 인원
    수업형태, 강의 언어 -> 비고에 포함
    '''
    lecture = dict(lecturelist.loc[i,['교과구분',
                                      '개설대학', 
                                      '개설학과', 
                                      '이수과정', 
                                      '학년', 
                                      '교과목번호', 
                                      '강좌번호', 
                                      '교과목명', 
                                      '학점', 
                                      '강의', 
                                      '실습' , 
                                      '수업교시', 
                                      '수업형태', 
                                      '강의실(동-호)(#연건, *평창)', 
                                      '주담당교수', 
                                      '장바구니신청', 
                                      '정원', 
                                      '비고', 
                                      '강의언어']])
    
    # key를 django field에 맞게 수정
    reviseKey(lecture)
        
    # NaN값은 파이썬이 인식 못 하므로 삭제
    NaNtoNone(lecture)
    
    # int 값들 변환, times는 아래에서 따로 처리
    if lecture['grade'] is not None:
        lecture['grade'] = int(lecture['grade'][0])
    lecture['lecture_code'] = int(lecture['lecture_code'])
    lecture['credits'] = int(lecture['credits'])
    lecture['lecture'] = int(lecture['lecture'])
    lecture['laboratory'] = int(lecture['laboratory'])
    lecture['cart'] = int(lecture['cart'])
    lecture['quota'] = int(lecture['quota'].split(' ')[0])
    
    # 언어와 형태 비고에 삽입
    reviseRemark(lecture)
    
    # 수업 교시 list로 변환
    times = lecture.pop('times')
    locations = lecture.pop('locations')
    if times is not None:
        times = [(time[0], int(time[2:4]+time[5:7]), int(time[8:10]+time[11:13])) for time in times.split('/')]
        lecture['times'] = times
        if locations is not None:
            locations = locations.split('/')
            lecture['locations'] = locations
    else:
        lecture['times'] = []
    
    # 개설학과가 None 이면 개설학과 = 개설대학
    if lecture['department'] is None:
        lecture['department'] = lecture['college']
    
    lectures.append(lecture)
    
print('강의 수 :', len(lectures))

강의 수 : 8038


# 데이터 삽입

In [192]:
for lecture in lectures:
    # college 오브젝트 생성
    college = lecture.pop('college')
    college = College.objects.get_or_create(name=college)
    
    # department 오브젝트 생성
    dept = lecture.pop('college')
    dept = Department.objects.get_or_create(name=dept, college=college)
        
    # course 오브젝트 생성
    course = {}
    course['title'] = lecture.pop('title')
    course['instructor'] = lecture.pop('instructor')
    course = Course.objects.get_or_create(**course)
    
    # lecture 오브젝트 생성
    times = lecture.pop('times')
    lecture['semester'] = '2022년 1학기'
    Lecture.objects.create(**lecture)
    
    # lecturetime 오브젝트 생성
    for time in times:
        LectureTime.objects.create(lecture=lecture, day=time[0], start=time[1], end=time[2])

[('월', 1600, 1715), ('금', 1400, 1515)]

In [11]:
printTuple(['2022년 1학기', '2021년 겨울학기', '2021년 2학기', '2021년 여름학기', '2021년 1학기', '2020년 겨울학기', '2020년 2학기', '2020년 여름학기', '2020년 1학기'])

		('2022년 1학기', '2022년 1학기'),
		('2021년 겨울학기', '2021년 겨울학기'),
		('2021년 2학기', '2021년 2학기'),
		('2021년 여름학기', '2021년 여름학기'),
		('2021년 1학기', '2021년 1학기'),
		('2020년 겨울학기', '2020년 겨울학기'),
		('2020년 2학기', '2020년 2학기'),
		('2020년 여름학기', '2020년 여름학기'),
		('2020년 1학기', '2020년 1학기'),
